In [0]:
from pyspark.sql.functions import *
from delta.tables import *

In [0]:
dbutils.widgets.text('folder_name','')

In [0]:
folder_name = dbutils.widgets.get('folder_name')

In [0]:
folder_name

'factsales/'

In [0]:
df = spark.readStream.format('delta')\
    .load(f'abfss://silver@osspstorageaccount.dfs.core.windows.net/{folder_name}')


In [0]:
def merge_to_target(micro_batch_df: DataFrame, batch_id: int):
    target_path = f"abfss://gold@osspstorageaccount.dfs.core.windows.net/{folder_name}"
    merge_key = key

    # 1. Check if the target Delta table exists.
    if DeltaTable.isDeltaTable(spark, target_path):
        target_table = DeltaTable.forPath(spark, target_path)
        (target_table.alias("t")
            .merge(micro_batch_df.alias("s"), f"t.{merge_key} = s.{merge_key}")
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )
    else:
        micro_batch_df.write.format("delta").save(target_path)

In [0]:
if folder_name == 'customer/':
    key = 'CustomerKey'
    df = df.withColumn('sg_customerkey',(unix_timestamp(col('updated_at'))* 1000000) +col('CustomerKey'))
    df_customer = spark.read.format('delta').load(f'abfss://silver@osspstorageaccount.dfs.core.windows.net/customer')


In [0]:
if folder_name == 'date/':
    key = 'DateKey'
    df = df.withColumn('sg_datekey',(unix_timestamp(col('updated_at')) * 1000000) + col('DateKey'))

In [0]:
if folder_name == 'paymentmethod/':
    key = 'PaymentKey'
    df = df.withColumn('sg_paymentmethodkey',(unix_timestamp(col('updated_at'))*1000000) + col('PaymentKey'))

In [0]:
if folder_name == 'product/':
    key = 'ProductKey'
    df = df.withColumn('sg_productkey',(unix_timestamp(col('updated_at'))*1000000) + col('ProductKey'))

In [0]:
if folder_name == 'promotion/':
    key = 'PromotionKey'
    df = df.withColumn('sg_promotionkey',(unix_timestamp(col('updated_at'))*1000000) + col('PromotionKey'))


In [0]:
if folder_name == 'region/':
    key = 'RegionKey'
    df = df.withColumn('sg_regionkey',(unix_timestamp(col('updated_at'))*1000000) + col('RegionKey'))

In [0]:
if folder_name == 'shippingmethod/':
    key = 'ShippingKey'
    df = df.withColumn('sg_shippingmethodkey',(unix_timestamp(col('updated_at'))*1000000) + col('ShippingKey'))

In [0]:
if folder_name != 'factsales/':
    df.writeStream\
        .format('delta')\
        .foreachBatch(merge_to_target)\
        .outputMode('update')\
        .option('checkpointLocation', f'abfss://gold@osspstorageaccount.dfs.core.windows.net/checkpoints/{folder_name}')\
        .trigger(once=True).start()
else:
    if DeltaTable.isDeltaTable(spark, 'abfss://gold@osspstorageaccount.dfs.core.windows.net/customer') and DeltaTable.isDeltaTable(spark,'abfss://gold@osspstorageaccount.dfs.core.windows.net/date') and DeltaTable.isDeltaTable(spark,'abfss://gold@osspstorageaccount.dfs.core.windows.net/paymentmethod') and DeltaTable.isDeltaTable(spark,'abfss://gold@osspstorageaccount.dfs.core.windows.net/product') and DeltaTable.isDeltaTable(spark,'abfss://gold@osspstorageaccount.dfs.core.windows.net/promotion') and DeltaTable.isDeltaTable(spark,'abfss://gold@osspstorageaccount.dfs.core.windows.net/region') and DeltaTable.isDeltaTable(spark,'abfss://gold@osspstorageaccount.dfs.core.windows.net/shippingmethod'):
        dim_customer = spark.read.format('delta').load(f'abfss://gold@osspstorageaccount.dfs.core.windows.net/customer')
        dim_date = spark.read.format('delta').load(f'abfss://gold@osspstorageaccount.dfs.core.windows.net/date')
        dim_paymentmethod = spark.read.format('delta').load(f'abfss://gold@osspstorageaccount.dfs.core.windows.net/paymentmethod')
        dim_product = spark.read.format('delta').load(f'abfss://gold@osspstorageaccount.dfs.core.windows.net/product')
        dim_promotion = spark.read.format('delta').load(f'abfss://gold@osspstorageaccount.dfs.core.windows.net/promotion')
        dim_region = spark.read.format('delta').load(f'abfss://gold@osspstorageaccount.dfs.core.windows.net/region')
        dim_shippingmethod = spark.read.format('delta').load(f'abfss://gold@osspstorageaccount.dfs.core.windows.net/shippingmethod')
        df = (
        df
        .join(dim_customer, df.CustomerKey == dim_customer.CustomerKey, 'left')
        .join(dim_date, df.DateKey == dim_date.DateKey, 'left')
        .join(dim_paymentmethod, df.PaymentKey == dim_paymentmethod.PaymentKey, 'left')
        .join(dim_product, df.ProductKey == dim_product.ProductKey, 'left')
        .join(dim_promotion, df.PromotionKey == dim_promotion.PromotionKey, 'left')
        .join(dim_region, df.RegionKey == dim_region.RegionKey, 'left')
        .join(dim_shippingmethod, df.ShippingKey == dim_shippingmethod.ShippingKey, 'left')
    )
        df = df.select('SalesKey','sg_customerkey','sg_datekey','sg_paymentmethodkey','sg_productkey','sg_promotionkey','sg_regionkey','sg_shippingmethodkey','Quantity','UnitPrice','TotalRevenue')
        df = df.withColumnRenamed('SalesKey','sg_saleskey').withColumnRenamed('Quantity','quantity').withColumnRenamed('UnitPrice','unitprice').withColumnRenamed('TotalRevenue','totalrevenue')
        df.writeStream\
            .format('delta')\
            .outputMode('append')\
            .option('checkpointLocation', f'abfss://gold@osspstorageaccount.dfs.core.windows.net/checkpoints/{folder_name}')\
            .trigger(once=True).start('abfss://gold@osspstorageaccount.dfs.core.windows.net/factsales')